In [1]:
import random, os
import cv2
import numpy as np
import imutils
from imutils import contours
from skimage.filters import threshold_local

In [2]:
def resize_image(image):
    r = 500.0 / image.shape[1]
    dim = (500, int(image.shape[0] * r))
    kernel = np.ones((5,5),np.uint8)

    gray = cv2.GaussianBlur(image,(3,3),1)
    resized = cv2.bilateralFilter(gray, 9 , 50, 50)

    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    bin_image = cv2.adaptiveThreshold(gray,250,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,9,2)
    gray = cv2.morphologyEx(bin_image, cv2.MORPH_OPEN, kernel)
    gray = cv2.morphologyEx(bin_image, cv2.MORPH_CLOSE, kernel)
    #bin_image = cv2.adaptiveThreshold(gray,250,cv2.THRESH_BINARY,cv2.,9,2)


    #gray = cv2.dilate(gray, kernel, iterations = 1)
    #bin_image = cv2.adaptiveThreshold(gray,250,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,3,1)

    resized = cv2.resize(bin_image, dim, interpolation = cv2.INTER_AREA)
    
    return resized

In [3]:
def morph_canny(image):
    kernel = np.ones((5,5),np.uint16)

    #gray = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
    #gray = cv2.dilate(gray,kernel,iterations = 5)
    image = cv2.Canny(image, 50, 200)

    cv2.imshow("Edged", image)
    cv2.waitKey(0)
    return image

In [4]:
def contour_warp_corp(image1,image2):
    cnts = cv2.findContours(image1.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
    screenCnt = 0
 
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
        # if our approximated contour has four points, then we
        # can assume that we have found our screen
        if len(approx) == 4:
            screenCnt = approx
            break

    cv2.drawContours(image2, [screenCnt], -1, (0, 255, 0), 2)
    cv2.imshow("Outline", image2)
    cv2.waitKey(0)
    
    warped = four_point_transform(image2, screenCnt.reshape(4,2))
    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    T = threshold_local(warped, 11, offset = 10, method = 'gaussian')
    warped = (warped > T).astype("uint8") * 255

    cv2.imshow('Cropped',warped)
    cv2.waitKey(0)
    return warped

In [5]:
path = r'C:\Users\Lap Mania\DealSmash\Img-Issue'

In [18]:
random_filename1 = random.choice([
    x for x in os.listdir(path)

         if os.path.isfile(os.path.join(path, x))
])

In [19]:
img0 = cv2.imread(random_filename1)

In [20]:
img1 = resize_image(img0)

In [21]:
cv2.imshow('Gaussian', img1)
cv2.waitKey(0)

-1